In [23]:
import oda_api.token 
import logging
import numpy as np
from oda_api.api import DispatcherAPI
from oda_api.plot_tools import OdaImage, OdaLightCurve, OdaSpectrum
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from matplotlib.patches import Circle
from astropy.time import Time

In [24]:
# token = ''
# token = getpass.getpass('Insert the token')
token = oda_api.token.discover_token() # get token stored locally

found token in TokenLocation.FILE_CUR_DIR your token payload: {
    "email": "hucklethums@icloud.com",
    "exp": 1742905994,
    "name": "interstellxr",
    "roles": "authenticated user, public-pool-hpc",
    "sub": "hucklethums@icloud.com"
}
token expires in 333.7 h


In [25]:
logging.getLogger().setLevel(logging.INFO) # WARNING, INFO or DEBUG
logging.getLogger('oda_api').addHandler(logging.StreamHandler())

Import the ScWs from the saved file.

In [26]:
scw_ids = []
scw_versions = []
scw_start_times = []
scw_end_times = []
jupiter_ra = []
jupiter_dec = []

with open("../data/2004-01-01_2024-01-01.txt", "r") as f:
    next(f)
    for line in f:
        parts = line.strip().split(", ")
        scw_ids.append(parts[0])
        scw_versions.append(parts[1])
        scw_start_times.append(float(parts[2]))  
        scw_end_times.append(float(parts[3]))  
        jupiter_ra.append(float(parts[4]))  
        jupiter_dec.append(float(parts[5]))  

# Sort by time and remove duplicates
unique_sorted_data = {}
for sid, ver, start, end, ra, dec in sorted(zip(scw_ids, scw_versions, scw_start_times, scw_end_times, jupiter_ra, jupiter_dec), key=lambda x: x[0]):
    if sid not in unique_sorted_data:  
        unique_sorted_data[sid] = (sid, ver, start, end, ra, dec)

scw_ids, scw_versions, scw_start_times, scw_end_times, jupiter_ra, jupiter_dec = map(list, zip(*unique_sorted_data.values()))

Add IDs and versions together to make correctly formatted ScWs, and compute durations of the ScW observations.

In [27]:
scw_list = [id + "." + ver for id, ver in zip(scw_ids, scw_versions)]
scw_durations = [(end - start)*24*60 for start, end in zip(scw_start_times, scw_end_times)]
print(scw_list)
print(scw_durations)
print([Time(start, format='mjd').isot for start in scw_start_times])
print("Max duration in minutes:", np.max(scw_durations), "at position", np.argmax(scw_durations))

['026601100010.001', '026601110010.001', '026700000210.001', '026700000230.001', '043200640010.001', '043200650010.001', '091700010010.001', '091700020010.001', '106300180010.001', '106300190010.001', '112600220010.001', '112600230010.001', '124200010010.001', '124200020010.001', '168600570010.001', '168600580010.001', '169500010010.001', '169500020010.001', '230200070010.001', '230200080010.001', '230200090010.001', '230200160010.001', '230200170010.001', '230200180010.001', '230200470010.001', '230200480010.001', '230200490010.001', '230200550010.001', '230200560010.001', '230200570010.001', '230200570020.001', '236600410010.001', '236600420010.001', '236600440010.001', '236600450010.001', '236600460010.001', '236600470010.001', '236600480010.001', '236600490010.001', '236600500010.001', '236600510010.001', '236600520010.001', '236600530010.001', '236600540010.001', '236600550010.001', '236600560010.001', '236600570010.001', '236600580010.001', '236600590010.001', '236600600010.001',

Group the ScWs by revolution number (and pointing number).

In [41]:
from collections import defaultdict

scw_groups = defaultdict(list)

for idx, scw in enumerate(scw_list):
    R = scw[:4]  
    P = scw[4:8] 
    #scw_groups[(R, P)].append(idx)  
    scw_groups[R].append(idx)  

large_scw_groups = []

#for (R, P), indices in scw_groups.items():
for R, indices in scw_groups.items():
    if len(indices)>10: # select revolutions with many pointings
        #print(f"Group R={R}, P={P}: SCWs at positions {indices} -> {', '.join([scw_list[i] for i in indices])}")
        print(f"Group R={R}: SCWs at positions {indices} -> {', '.join([scw_list[i] for i in indices])}")
        large_scw_groups.append([i for i in indices])

# We found groups of ScWs with more than 10 consecutive pointings
# Let us check the dates and see they are really consecutive and how close together they are

group = large_scw_groups[0]
Revs = [R for R, indices in scw_groups.items() if len(indices)>10]
R = Revs[0]

print([scw_list[i] for i in group])
print([Time(scw_start_times[i], format='mjd').isot for i in group])
print([scw_durations[i] for i in group])
print(len(group))

Group R=2302: SCWs at positions [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30] -> 230200070010.001, 230200080010.001, 230200090010.001, 230200160010.001, 230200170010.001, 230200180010.001, 230200470010.001, 230200480010.001, 230200490010.001, 230200550010.001, 230200560010.001, 230200570010.001, 230200570020.001
Group R=2366: SCWs at positions [31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51] -> 236600410010.001, 236600420010.001, 236600440010.001, 236600450010.001, 236600460010.001, 236600470010.001, 236600480010.001, 236600490010.001, 236600500010.001, 236600510010.001, 236600520010.001, 236600530010.001, 236600540010.001, 236600550010.001, 236600560010.001, 236600570010.001, 236600580010.001, 236600590010.001, 236600600010.001, 236600610010.001, 236600620010.001
Group R=2367: SCWs at positions [52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74] -> 236700010010.001, 236700020010.001, 236700030010.001, 

Asynchronously query the ODA API with scw list and save the fits files.

In [ ]:
disp_by_scw = {}
data_by_scw = {}

while True:
    image_results = []

    for list_position in group:
        ra_deg = jupiter_ra[list_position]
        dec_deg = jupiter_dec[list_position]

        par_dict = {
            "E1_keV": "15",
            "E2_keV": "30",
            "T_format": "isot",
            "detection_threshold": "5",
            "instrument": "isgri",
            "osa_version": "OSA11.2",
            "product": "isgri_image",
            "product_type": "Real",
            "scw_list": [scw_list[list_position]],
            'token': token
        }
        
        scw_id = scw_list[list_position]

        if scw_id not in disp_by_scw:
            disp_by_scw[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock", wait=False)
        
        _disp = disp_by_scw[scw_id]
        
        data = data_by_scw.get(scw_id, None)

        if data is None and not _disp.is_failed:
            if not _disp.is_submitted:
                data = _disp.get_product(**par_dict)
            else:
                _disp.poll()

            print("Is complete ", _disp.is_complete)
            if not _disp.is_complete:
                continue
            else:
                data = _disp.get_product(**par_dict)
                data_by_scw[scw_id] = data

        image_results.append(data)


In [48]:
import os
output_dir = f"../data/Rev_{R}/"
os.makedirs(output_dir, exist_ok=True)
for i, image in enumerate(image_results):
    im = OdaImage(image)
    file_path = os.path.join(output_dir, f"{scw_ids[i]}.{scw_versions[i]}")
    im.write_fits(file_path)

Plot from the fits file and extract countrates/uncertainties.

In [ ]:
import os

countrates = []
variances = []
exposuretimes = []
calculated_stds = []

for list_position in group:

    fits_file = f"../data/{scw_list[list_position]}"+"mosaic.fits"

    if not os.path.exists(fits_file):
        print(f"File not found: {fits_file}, skipping...")
        continue  

    pick = 2  # 2 for image, 3 for variance, 4 for significance and 5 for exposure map

    ra_deg = jupiter_ra[list_position]
    dec_deg = jupiter_dec[list_position]

    with fits.open(fits_file) as hdul:
        #hdul.info()
        intensity_data = hdul[2].data 
        variance_data = hdul[3].data
        significance_data = hdul[4].data
        exposure_data = hdul[5].data
        #print("\n")
        #print(repr(hdul[pick].header))
        #print("\n")
        #print(hdul[1].data.columns)
        wcs = WCS(hdul[pick].header)  
        CD1 = np.abs(hdul[pick].header["CD1_1"])

    data = [intensity_data, variance_data, significance_data, exposure_data]
    image_data = data[pick-2]

    x, y = wcs.all_world2pix(ra_deg, dec_deg, 1)
    x_int, y_int = int(round(x.item())), int(round(y.item()))

    countrates.append(intensity_data[y_int, x_int])
    variances.append(variance_data[y_int, x_int])
    exposuretimes.append(exposure_data[y_int, x_int])

    box_width, box_height = 4, 4

    y_min = max(0, y_int - box_height // 2)
    y_max = min(intensity_data.shape[0], y_int + box_height // 2)
    x_min = max(0, x_int - box_width // 2)
    x_max = min(intensity_data.shape[1], x_int + box_width // 2)
    box_region = intensity_data[y_min:y_max, x_min:x_max]

    calculated_stds.append(np.std(box_region))

    #'''
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection=wcs)
    im = ax.imshow(np.log10(image_data), cmap='plasma', origin='lower')

    ax.set_xlabel('RA')
    ax.set_ylabel('DEC')

    pixel_scale = CD1 * 3600  # pixel size in arcseconds (|CD1_1| from fits header)
    circle_radius_pixels = 30 * 40 / pixel_scale / 2  # radius in pixels (30 times Jupiter's apparent size which is ~ 40")

    circle = Circle((x, y), circle_radius_pixels, edgecolor='black', facecolor='none', lw = 0.5)
    ax.add_patch(circle)

    title = ""
    if pick == 2:
        title = "Intensity"
    elif pick == 3:
        title = "Variance"
    elif pick == 4:
        title = "Significance"
    elif pick == 5:
        title = "Exposure"

    plt.title(title+" map")
    plt.colorbar(im, label="Intensity")

    ax.annotate(
        text="Jupiter (30x Rj)", 
        xy=(x, y),  
        xytext=(x - 100, y + 150),
        arrowprops=dict(arrowstyle="->", color="black", lw=0.5),
        fontsize=11,
        color="black"
    )

    #plt.savefig(f"../data/{scw_list[list_position]}"+"_"+f"{title}.pdf", format='pdf', bbox_inches='tight')
    #'''

uncertainties = np.sqrt(variances)


File not found: ../data/230200070010.001mosaic.fits, skipping...
File not found: ../data/230200080010.001mosaic.fits, skipping...
File not found: ../data/230200090010.001mosaic.fits, skipping...
File not found: ../data/230200160010.001mosaic.fits, skipping...
File not found: ../data/230200170010.001mosaic.fits, skipping...
File not found: ../data/230200180010.001mosaic.fits, skipping...
File not found: ../data/230200470010.001mosaic.fits, skipping...
File not found: ../data/230200480010.001mosaic.fits, skipping...
File not found: ../data/230200490010.001mosaic.fits, skipping...
File not found: ../data/230200550010.001mosaic.fits, skipping...
File not found: ../data/230200560010.001mosaic.fits, skipping...
File not found: ../data/230200570010.001mosaic.fits, skipping...
File not found: ../data/230200570020.001mosaic.fits, skipping...


In [ ]:
isot_start_times = [Time(scw_start_times[i], format='mjd').isot for i in group]
isot_end_times = [Time(scw_end_times[i], format='mjd').isot for i in group]

exposure_durations = np.array(scw_end_times) - np.array(scw_start_times)

isot_mid_times = Time((np.array(scw_start_times) + np.array(scw_end_times)) / 2, format='mjd').plot_date

plt.figure(figsize=(10, 6))

plt.bar(isot_mid_times, countrates, width=exposure_durations, color='b', alpha=0.6, edgecolor='black', label='Intensity')
plt.errorbar(isot_mid_times, countrates, yerr=uncertainties, fmt='o', color='r', capsize=5, label='Error bars')

plt.xlabel('Time', fontsize=14)
plt.ylabel('Countrate (counts/s)', fontsize=14)
plt.title('Intensity over Time (SCWs)', fontsize=16)
plt.grid(True)
plt.legend()

import matplotlib.dates as mdates
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.xticks(rotation=45)
plt.show()


In [ ]:
plt.figure()
plt.scatter(uncertainties, calculated_stds)
plt.xlabel('Calculated STDevs', fontsize=14)
plt.ylabel('Extracted STDevs', fontsize=14)
plt.title('True vs calculated uncertainties', fontsize=16)
plt.grid(True)
plt.legend()